<a href="https://colab.research.google.com/github/leksandre/-crab-for-web2ipr_book_shop/blob/main/gDriveToMega_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# //!pip3 install mega
# //!pip install mega.py==0.8.5
!pip install mega.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotly 5.24.1 requires tenacity>=6.2.0, but you have tenacity 5.1.5 which is incompatible.


In [2]:
from mega import Mega
import os

In [3]:
%mkdir /content/tmp1

In [12]:
from re import S
def human_size(bytes, units=[' bytes','KB','MB','GB','TB', 'PB', 'EB']):
    """ Returns a human readable string representation of bytes """
    return str(bytes) + units[0] if bytes < 1024 else human_size(bytes>>10, units[1:])

from pathlib import Path
from os.path import isdir
import shutil
errors = {}
class UploadMega():
    def __init__(self, id, password):
        mega = Mega()
        self.mega = mega.login(id, password)
    def upload_single_file(self, path):
        print("uploading :", path)
        self.mega.upload(path)
        print("uploaded :", path)

    def compare_dir(self, path, newFolder = ''):
        files = os.listdir(path)
        j = 0
        for i in files:
            if isdir(i):
              if i == 'Google Фото':
                continue
              print("\n\ngo to DIR :", i)
              l = self.mega.find(i)
              if l:
                print(' exist DIR ', newFolder+ "/"+i , l)
                self.compare_dir(os.path.join(path, i), newFolder+'/'+i)
              else:
                print("have to create DIR:", i)
            else:
              # print("go to file :", newFolder, " / ",i)
              l = self.mega.find(i)
              # print('check exist file ', newFolder+ "/"+i , l)
              if l:
                # print('already exist file', i)
                j = j+1

                hasErr = False
                s = (os.path.getsize(os.path.join(path, i)))
                if l[1]['s'] != s:
                  print('!!!!!----wrong file size 1', i, l[1]['s'], "<>", s )
                  print(human_size(l[1]['s']))
                  hasErr = True

                s = (os.stat(os.path.join(path, i)).st_size )
                if l[1]['s'] != s:
                  print('!!!!!----wrong file size 2', i, l[1]['s'], "<>", s )
                  print(human_size(l[1]['s']))
                  hasErr = True

                s = (Path(os.path.join(path, i)).stat().st_size)
                if l[1]['s'] != s:
                  print('!!!!!----wrong file size 3', i, l[1]['s'], "<>", s )
                  print(human_size(l[1]['s']))
                  hasErr = True

                if hasErr:
                  m.delete(l[0])
                  print("----------replace----------------have to upload file:", newFolder+ "/"+i)
                  Folder = self.mega.find(newFolder)
                  try:
                    self.mega.upload(os.path.join(path, i), Folder[0])
                  except KeyError as e:
                    print("Exception -------replace-------- ", newFolder, " / ",i)
                    print(' over KeyError  ' + str(e))
                  except Exception as e:
                    print("Exception -------replace-------- ", newFolder, " / ",i)
                    print(' over KeyError Exception  ' + str(e))

                # print(j ,l,' exist file ', newFolder+ "/"+i , l)

                # print(l[1]['s'])
                # print(human_size(l[1]['s']))

                # print(os.path.getsize(os.path.join(path, i)))
                # print(os.stat(os.path.join(path, i)).st_size )
                # print(Path(os.path.join(path, i)).stat().st_size)

                pass
              else:
                print("--------------------------have to upload file:", newFolder+ "/"+i)
                Folder = self.mega.find(newFolder)
                try:
                  self.mega.upload(os.path.join(path, i), Folder[0])
                except KeyError as e:
                    # print("Exception --------------- ", newFolder, " / ",i)
                    print(' over KeyError  ' + str(e))
                except Exception as e:
                    # print("Exception --------------- ", newFolder, " / ",i)
                    print(' over KeyError Exception  ' + str(e))

                    file = os.path.join(path, i)

                    filename, file_extension = os.path.splitext(file)
                    if file_extension not in errors:
                      errors[file_extension] = 0
                    errors[file_extension] = errors[file_extension] + 1
                    # try:
                    #     shutil.copyfile(file, '/content/tmp1')
                    # except Exception as e:
                    #     # print("Exception -------shutil-------- ", newFolder, " / ",i)
                    #     print(' over KeyError Exception  ' + str(e))

                    # try:
                    #
                    #   %cp "{file}" /content/tmp1
                    # except Exception as e:
                    #     # print("Exception -------cp-------- ", newFolder, " / ",i)
                    #     print(' over KeyError Exception  ' + str(e))


        print(j,' end path ',path)

    def upload_directory(self, path, newFolder = ''):
        files = os.listdir(path)
        for i in files:
            if isdir(i):
              if i == 'Google Фото':
                continue
              print("uploading DIR :", i)
              # if len(newFolder)>0:
              l = self.mega.find(i)
              print(' exist DIR ', newFolder+ "/"+i , l)
              if l:
                print('already exist')
              else:
                self.mega.create_folder(newFolder+'/'+i)
              self.upload_directory(os.path.join(path, i), newFolder+'/'+i)
              print("uploaded DIR:", i)
            else:
              print("uploading :", newFolder, " / ",i)
              l = self.mega.find(i)
              print(' exist file ', newFolder+ "/"+i , l)
              if l:
                print('already exist')
                continue
              Folder = self.mega.find(newFolder)
              try:
                self.mega.upload(os.path.join(path, i), Folder[0])
              except Exception:
                print("Exception --------------- ", newFolder, " / ",i)
              print("uploaded :", i)

    def upload_files_by_extension(self, path, extension):
        files = os.listdir(path)
        for i in files:
            if i.split(".")[-1] == extension:
                print("uploading :", i)
                self.mega.upload(os.path.join(path, i))
                print("uploaded :", i)

print(errors)

{}


In [13]:
m = UploadMega();

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
%cd /content/drive/My Drive/mega2/
# %ls

/content/drive/My Drive/mega2


In [16]:
import os, os.path

# simple version for working with CWD
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

# path joining version for other paths
DIR = '/content/drive/My Drive/mega2/'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

398
398


In [17]:
files = m.mega.get_files()

In [ ]:
print(files)

In [ ]:
 m.compare_dir('/content/drive/My Drive/mega2/','mega2')

In [20]:
# m.upload_directory('/content/drive/My Drive/mega2/', 'mega2')

In [22]:
print(errors)

{'.gdoc': 75, '.gsheet': 59, '.gslides': 7, '.zip': 3, '.pt': 18, '.gz': 1, '': 1}


In [21]:
print('success')

success


In [33]:
files11 = m.mega.get_files()

In [ ]:
print(files11)

In [ ]:
for i in range(0,10):
  str1="("+str(i)+")"
  print(str1)
  for key in files11:
    name1 = files11[key]['a']['n']
    if name1.find(str1)>-1:
      print(key, "->", name1)
      l = m.mega.find(name1)
      newName1 = name1.replace(str1,'')
      if l:
        print(' rename ', name1,"->",newName1)
        m.mega.rename(l, newName1)

In [ ]:
files11 = m.mega.get_files()
str1 = " ."
for key in files11:
    name1 = files11[key]['a']['n']
    if name1.find(str1)>-1:
      # print(key, "->", name1)
      newName2 = name1.replace(str1,'.')
      s1 = files11[key]['s']
      for key2 in files11:
        name2 = files11[key2]['a']['n']
        if name2==newName2:
          s2 = files11[key2]['s']
          if s2!=s1:
            print(' compare ', newName2,"(",s2,")<->",name1,"(",s1,")")
          else:
            l = m.mega.find(name1)
            if l:
              print(' rename ', name1,"->",newName2)
              m.mega.rename(l, newName1)